In [1]:
import pandas as pd
import pandas as pd
from collections import defaultdict
from sklearn import metrics
import numpy as np
import sys
import time
import os

start_time = time.time()

inbinfile='/Users/irffanalahi/Research/Research_update/SoftRD/reproduce_goodmelresult_withHardRC_new_implementation/AUC1fullcohort/BL22_TILAUC1_dummyfull.txt_result_dupindex_binnedstats.pkl'


outfolder=inbinfile+"_HardRC"
os.mkdir(outfolder)

outfile=outfolder+"/"+os.path.basename(inbinfile)


#######outfile=inbinfile


mincpgLIST=[1,2,3,4,5]

consideringALLheyper=True


inbindf=pd.read_pickle(inbinfile)



#scorecolumns=['CD4-others','CD8-others','nB-others','NK-others','Mn-others','mNeu-others','m8-others','DC-others','Eo-others','Tr-others','m4-others','Mg-others','Er-others','mB-others']


#13ctref
#scorecolumns=['CD4-others','CD8-others','nB-others','NK-others','PC-others','Mn-others','iDC-others','mDC-others','M1-others','M0-others','mN-others','M2-others','cB-others']


#BL22LTME
scorecolumns=['NaiveCD4-others','NaiveCD8-others','nB-others','NK-others','PC-others','Mono-others','M0-others','M1-others','M2-others','iDC-others','mDC-others','PMN-others','cm8-others','em8-others','Eo-others','Tregs-others','em4-others','ed8-others','Mg-others','cm4-others','Er-others','mB-others','TIL-others','MelTumor-others']


#old 3 compartment LTME good SM
#scorecolumns=['PBL-others','TIL-others','MEL_TUMOR-others']



inbindf=inbindf.reset_index()
inbindf.head()

,index,NaiveCD4-others,NaiveCD8-others,nB-others,NK-others,PC-others,Mono-others,M0-others,M1-others,M2-others,...,LENhypoCpG,LENhyperCpG,total_cpg,LENhypoCpG_BY_total_cpg,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename
0,A00354:161:HVKCVDSXX:2:1419:24831:1846,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,...,1.0,4.0,5.0,0.200000,TIL-others,0.512984,TIL-others,0.102597,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...
1,A00354:161:HVKCVDSXX:3:1637:15230:24831,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,...,1.0,6.0,7.0,0.142857,TIL-others,0.594756,TIL-others,0.084965,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...
2,A00354:161:HVKCVDSXX:3:2571:21929:24533,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,...,1.0,2.0,3.0,0.333333,TIL-others,0.620848,TIL-others,0.206949,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...
3,A00354:161:HVKCVDSXX:4:2573:21630:4539,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,...,2.0,1.0,3.0,0.666667,TIL-others,1.154968,TIL-others,0.769979,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...
4,A00354:161:HVKCVDSXX:1:1627:30110:5196,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,0.0,1.0,1.0,0.000000,TIL-others,0.607206,NaiveCD4-others,-0.000000,NotAssigned,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...


In [2]:
if consideringALLheyper==True:
    allassigned=inbindf.copy()


elif consideringALLheyper==False:
    allassigned=inbindf[inbindf['deltabasedfragassignment']!='NotAssigned'].copy()
    
else:
    print("wrong input")
    sys.exit(1)
#
allassigned.head()

,index,NaiveCD4-others,NaiveCD8-others,nB-others,NK-others,PC-others,Mono-others,M0-others,M1-others,M2-others,...,LENhypoCpG,LENhyperCpG,total_cpg,LENhypoCpG_BY_total_cpg,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename
0,A00354:161:HVKCVDSXX:2:1419:24831:1846,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,...,1.0,4.0,5.0,0.200000,TIL-others,0.512984,TIL-others,0.102597,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...
1,A00354:161:HVKCVDSXX:3:1637:15230:24831,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,...,1.0,6.0,7.0,0.142857,TIL-others,0.594756,TIL-others,0.084965,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...
2,A00354:161:HVKCVDSXX:3:2571:21929:24533,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,...,1.0,2.0,3.0,0.333333,TIL-others,0.620848,TIL-others,0.206949,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...
3,A00354:161:HVKCVDSXX:4:2573:21630:4539,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,...,2.0,1.0,3.0,0.666667,TIL-others,1.154968,TIL-others,0.769979,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...
4,A00354:161:HVKCVDSXX:1:1627:30110:5196,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,0.0,1.0,1.0,0.000000,TIL-others,0.607206,NaiveCD4-others,-0.000000,NotAssigned,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...


In [3]:
allassigned.loc[inbindf['deltabasedfragassignment']=='NotAssigned','deltabasedfragassignment']=allassigned.loc[inbindf['deltabasedfragassignment']=='NotAssigned','maxscoredCT_beforeCpGweight']

In [4]:
allassigned['secondmaxScore']='Notassigned'
allassigned['secondmaxScoreCT']='Notassigned'
allassigned.head()

,index,NaiveCD4-others,NaiveCD8-others,nB-others,NK-others,PC-others,Mono-others,M0-others,M1-others,M2-others,...,total_cpg,LENhypoCpG_BY_total_cpg,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,secondmaxScore,secondmaxScoreCT
0,A00354:161:HVKCVDSXX:2:1419:24831:1846,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,...,5.0,0.200000,TIL-others,0.512984,TIL-others,0.102597,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,Notassigned,Notassigned
1,A00354:161:HVKCVDSXX:3:1637:15230:24831,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,...,7.0,0.142857,TIL-others,0.594756,TIL-others,0.084965,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,Notassigned,Notassigned
2,A00354:161:HVKCVDSXX:3:2571:21929:24533,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,...,3.0,0.333333,TIL-others,0.620848,TIL-others,0.206949,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,Notassigned,Notassigned
3,A00354:161:HVKCVDSXX:4:2573:21630:4539,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,...,3.0,0.666667,TIL-others,1.154968,TIL-others,0.769979,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,Notassigned,Notassigned
4,A00354:161:HVKCVDSXX:1:1627:30110:5196,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,1.0,0.000000,TIL-others,0.607206,NaiveCD4-others,-0.000000,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,Notassigned,Notassigned


In [5]:
grouped=allassigned.groupby('maxscoredCT')

for name, group in grouped:
    
    
    tempscorecolumns=scorecolumns.copy()
    
    tempscorecolumns.remove(name)
    
    allassigned.loc[group.index,'secondmaxScore']=group[tempscorecolumns].max(axis=1)
    allassigned.loc[group.index,'secondmaxScoreCT']=(group[tempscorecolumns]).idxmax(axis=1)

In [6]:
allassigned['adjustedScore_minus']=allassigned['maxscore']-allassigned['secondmaxScore']
allassigned['adjustedScore_minus_avg']=(allassigned['maxscore']+(allassigned['maxscore']-allassigned['secondmaxScore']))/2
allassigned.head()

,index,NaiveCD4-others,NaiveCD8-others,nB-others,NK-others,PC-others,Mono-others,M0-others,M1-others,M2-others,...,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,secondmaxScore,secondmaxScoreCT,adjustedScore_minus,adjustedScore_minus_avg
0,A00354:161:HVKCVDSXX:2:1419:24831:1846,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,...,TIL-others,0.512984,TIL-others,0.102597,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0.02,NaiveCD4-others,0.122597,0.112597
1,A00354:161:HVKCVDSXX:3:1637:15230:24831,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,...,TIL-others,0.594756,TIL-others,0.084965,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0.0142857,NaiveCD4-others,0.0992509,0.092108
2,A00354:161:HVKCVDSXX:3:2571:21929:24533,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,...,TIL-others,0.620848,TIL-others,0.206949,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0.0333333,NaiveCD4-others,0.240283,0.223616
3,A00354:161:HVKCVDSXX:4:2573:21630:4539,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,...,TIL-others,1.154968,TIL-others,0.769979,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0.133333,NaiveCD4-others,0.903312,0.836645
4,A00354:161:HVKCVDSXX:1:1627:30110:5196,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,TIL-others,0.607206,NaiveCD4-others,-0.000000,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0,NaiveCD8-others,0,0


In [7]:
allassigned.columns

Index(['index', 'NaiveCD4-others', 'NaiveCD8-others', 'nB-others', 'NK-others',
       'PC-others', 'Mono-others', 'M0-others', 'M1-others', 'M2-others',
       'iDC-others', 'mDC-others', 'PMN-others', 'cm8-others', 'em8-others',
       'Eo-others', 'Tregs-others', 'em4-others', 'ed8-others', 'Mg-others',
       'cm4-others', 'Er-others', 'mB-others', 'TIL-others', 'MelTumor-others',
       'LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg',
       'maxscoredCT_beforeCpGweight', 'maxscore_beforeCpGweight',
       'maxscoredCT', 'maxscore', 'deltabasedfragassignment', 'filename',
       'secondmaxScore', 'secondmaxScoreCT', 'adjustedScore_minus',
       'adjustedScore_minus_avg'],
      dtype='object')

In [8]:
adjustedScore_minus0val=list(set(allassigned[allassigned['LENhypoCpG_BY_total_cpg']==0]['adjustedScore_minus'].tolist()))
if len(adjustedScore_minus0val)==0:
    pass
elif len(adjustedScore_minus0val)!=1 or adjustedScore_minus0val[0]!=0:
    print(adjustedScore_minus0val)
    sys.exit(1)
    

In [9]:
adjustedScore_minus_avg0val=list(set(allassigned[allassigned['LENhypoCpG_BY_total_cpg']==0]['adjustedScore_minus_avg'].tolist()))


if len(adjustedScore_minus_avg0val)==0:
    pass

elif len(adjustedScore_minus_avg0val)!=1 or adjustedScore_minus_avg0val[0]!=0:
    print(adjustedScore_minus_avg0val)
    sys.exit(1)
    

In [10]:
allassigned['LENhypoCpG_0.8']=0
allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.8,'LENhypoCpG_0.8']=allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.8,'LENhypoCpG']

In [11]:
allassigned['LENhypoCpG_0.65']=0
allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.65,'LENhypoCpG_0.65']=allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.65,'LENhypoCpG']

In [12]:
allassigned['LENhypoCpG_0.5']=0
allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.5,'LENhypoCpG_0.5']=allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.5,'LENhypoCpG']

In [13]:
allassigned.head()

,index,NaiveCD4-others,NaiveCD8-others,nB-others,NK-others,PC-others,Mono-others,M0-others,M1-others,M2-others,...,maxscore,deltabasedfragassignment,filename,secondmaxScore,secondmaxScoreCT,adjustedScore_minus,adjustedScore_minus_avg,LENhypoCpG_0.8,LENhypoCpG_0.65,LENhypoCpG_0.5
0,A00354:161:HVKCVDSXX:2:1419:24831:1846,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000,...,0.102597,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0.02,NaiveCD4-others,0.122597,0.112597,0.0,0.0,0.0
1,A00354:161:HVKCVDSXX:3:1637:15230:24831,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,-0.014286,...,0.084965,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0.0142857,NaiveCD4-others,0.0992509,0.092108,0.0,0.0,0.0
2,A00354:161:HVKCVDSXX:3:2571:21929:24533,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,...,0.206949,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0.0333333,NaiveCD4-others,0.240283,0.223616,0.0,0.0,0.0
3,A00354:161:HVKCVDSXX:4:2573:21630:4539,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,...,0.769979,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0.133333,NaiveCD4-others,0.903312,0.836645,0.0,2.0,2.0
4,A00354:161:HVKCVDSXX:1:1627:30110:5196,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,-0.000000,TIL-others,Y-3694-cfDNA_sorted_onlyregionIN_tilpercentile...,-0,NaiveCD8-others,0,0,0.0,0.0,0.0


In [14]:
def hardRC(fgrouped,targetmincpg,cpgpatternratecolumn,fname):
    ctproportiondict=defaultdict(list)
    for score in scorecolumns:
        
     
        
        totalfragments=fgrouped[(fgrouped['deltabasedfragassignment']==score) & (fgrouped['total_cpg']>=targetmincpg)].shape[0]
        positivefragments=fgrouped[(fgrouped['deltabasedfragassignment']==score) & (fgrouped[cpgpatternratecolumn]>=targetmincpg)].shape[0]
        

        

        
        if totalfragments==0:
            tempctproportion=0
        
        else:
            tempctproportion=1.0*positivefragments/totalfragments
        
        ctname=score.replace('-others','')
        
        ctproportiondict[ctname].append(tempctproportion)
        
    ctproportionDF=pd.DataFrame.from_dict(ctproportiondict)
    
    ctproportionDF['Mixture']=fname
        
    return  ctproportionDF
    
  

In [15]:
filegrouped=allassigned.groupby('filename')

for mincpg in mincpgLIST:
    allfileresultfiftypercent=[]
    allfileresultsixtyfivepercent=[]
    allfileresulteightypercent=[]
    for name, group in filegrouped:
        allfileresultfiftypercent.append(hardRC(group,mincpg,'LENhypoCpG_0.5',name))
        allfileresultsixtyfivepercent.append(hardRC(group,mincpg,'LENhypoCpG_0.65',name))
        allfileresulteightypercent.append(hardRC(group,mincpg,'LENhypoCpG_0.8',name))
        
        
        
     

    allfileresultfiftypercentDF=pd.concat(allfileresultfiftypercent)
    allfileresultfiftypercentDF.set_index('Mixture',inplace=True) 
    
    allfileresultsixtyfivepercentDF=pd.concat(allfileresultsixtyfivepercent)
    allfileresultsixtyfivepercentDF.set_index('Mixture',inplace=True) 
    
    
    allfileresulteightypercentDF=pd.concat(allfileresulteightypercent)
    allfileresulteightypercentDF.set_index('Mixture',inplace=True)
    
    
    
    allfileresultfiftypercentDF.to_csv(outfile+"_pattern"+".5_mincpg"+str(mincpg)+"_CSxOut.txt",sep="\t")
    
    allfileresultsixtyfivepercentDF.to_csv(outfile+"_pattern"+".65_mincpg"+str(mincpg)+"_CSxOut.txt",sep="\t")
    
    allfileresulteightypercentDF.to_csv(outfile+"_pattern"+".8_mincpg"+str(mincpg)+"_CSxOut.txt",sep="\t")
    



In [16]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

37.04119873046875
